In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

### Clients

In [2]:
from decouple import config


HPC_USER = config('HPC_USER')
HPC_HOST = config('HPC_HOST')
HPC_PASSPHRASE = config('HPC_PASSPHRASE')

In [3]:
from pathlib import Path

from math_rag.infrastructure.clients import HPCClient, PBSProClient, SFTPClient


sftp_client = SFTPClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)
pbs_pro_client = PBSProClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)
hpc_client = HPCClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('../assets/huggingface/tgi/tgi.sh')
target = Path('/lustre/home/lpanic/tgi_test.sh')

await sftp_client.upload(source, target)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('/lustre/home/lpanic/tgi_test.sh')
target = Path('../tmp/tgi.sh')

await sftp_client.download(source, target)

In [ ]:
# limited to 80 characters (79 + newline)
len('    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world')

79

In [6]:
# qstat -f 465317
queue_status = """
Job Id: 465317.x3000c0s25b0n0.hsn.hpc.srce.hr
    Job_Name = hello
    Job_Owner = lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr
    resources_used.cpupercent = 0
    resources_used.cput = 00:00:00
    resources_used.mem = 0b
    resources_used.ncpus = 1
    resources_used.vmem = 0kb
    resources_used.walltime = 00:00:00
    job_state = R
    queue = cpu-single
    server = x3000c0s25b0n0.hsn.hpc.srce.hr
    Checkpoint = u
    ctime = Fri Mar 28 10:45:48 2025
    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world
	/hello.e465317
    exec_host = x8000c1s4b0n1/1
    exec_vnode = (x8000c1s4b0n1:mem=1843200kb:ncpus=1:ngpus=0)
    Hold_Types = n
    Join_Path = n
    Keep_Files = oed
    Mail_Points = a
    mtime = Fri Mar 28 10:45:53 2025
    Output_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_worl
	d/hello.o465317
    Priority = 0
    qtime = Fri Mar 28 10:45:48 2025
    Rerunable = True
    Resource_List.mem = 1800mb
    Resource_List.ncpus = 1
    Resource_List.ngpus = 0
    Resource_List.nodect = 1
    Resource_List.place = pack
    Resource_List.select = 1:mem=1800mb:ncpus=1:ngpus=0
    Resource_List.walltime = 48:00:00
    stime = Fri Mar 28 10:45:48 2025
    session_id = 3900810
    jobdir = /lustre/home/lpanic
    substate = 42
    Variable_List = PBS_O_HOME=/lustre/home/lpanic,PBS_O_LANG=en_US.UTF-8,
	PBS_O_LOGNAME=lpanic,
	PBS_O_PATH=/lustre/home/lpanic/.local/bin:/lustre/home/lpanic/bin:/opt
	/clmgr/sbin:/opt/clmgr/bin:/opt/sgi/sbin:/opt/sgi/bin:/usr/share/Module
	s/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/c3/bin:/op
	t/pbs/bin:/sbin:/bin,PBS_O_MAIL=/var/spool/mail/lpanic,
	PBS_O_SHELL=/bin/bash,PBS_O_HOST=x3000c0s27b0n0.hsn.hpc.srce.hr,
	PBS_O_WORKDIR=/lustre/home/lpanic/hello_world,PBS_O_SYSTEM=Linux,
	PBS_O_QUEUE=RouteQ
         = Job run at Fri Mar 28 at 10:45 on (x8000c1s4b0n1:mem=1843200kb:nc
	pus=1:ngpus=0)
    etime = Fri Mar 28 10:45:48 2025
    run_count = 1
    eligible_time = 00:00:05
    Submit_arguments = -koed hello.sh
    project = _pbs_project_default
    Submit_Host = x3000c0s27b0n0.hsn.hpc.srce.hr
"""

In [7]:
from math_rag.infrastructure.mappings.hpcs.pbs import PBSProJobFullMapping


job = PBSProJobFullMapping.to_source(queue_status)

In [5]:
pbs_path = Path('hello_world/hello2b.sh')
job_id = await pbs_pro_client.queue_submit(pbs_path)
job_id

'469063.x3000c0s25b0n0.hsn.hpc.srce.hr'

In [8]:
job = await pbs_pro_client.queue_status(job_id, alternate=True, full=False)
job

PBSProJobAlternate(id='469063.x3000c0s25b0*', user='lpanic', queue=<HPCQueue.GPU: 'gpu'>, name='hello', session_id=None, num_chunks=1, num_cpus=1, requested_mem=120000000000, requested_time=datetime.timedelta(seconds=600), state=<PBSProJobState.QUEUED: 'Q'>, elapsed_time=None)

In [10]:
job = await pbs_pro_client.queue_status(job_id, alternate=False, full=True)
job

PBSProJobFull(id='469063.x3000c0s25b0n0.hsn.hpc.srce.hr', name='hello', owner='lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr', state=<PBSProJobState.QUEUED: 'Q'>, queue=<HPCQueue.GPU: 'gpu'>, server='x3000c0s25b0n0.hsn.hpc.srce.hr', checkpoint='u', exec_host=None, exec_vnode=None, error_path='x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/error.log', output_path='x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/output.log', dir=None, hold_types=<PBSProHoldType.NONE: 'n'>, join_path=<PBSProJoinPath.NONE: 'n'>, keep_files=<PBSProKeepFiles.STDOUT_STDERR_DIRECT: 'oed'>, mail_points=<PBSProMailPoints.ABORT: 'a'>, substate=10, priority=0, session_id=None, rerunable=True, run_count=None, submit_arguments='-koed hello_world/hello2b.sh', project='_pbs_project_default', submit_host='x3000c0s27b0n0.hsn.hpc.srce.hr', created=datetime.datetime(2025, 3, 31, 14, 12, 5), queued=datetime.datetime(2025, 3, 31, 14, 12, 5), modified=datetime.datetime(2025, 3, 31, 14, 12, 5), started=None, eligible=datetime.

In [ ]:
await pbs_pro_client.trace_job(job_id)

In [12]:
await pbs_pro_client.queue_delete(job_id, force=False)

### Prepare input file

In [2]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

TGI_BASE_URL = ''
TGI_MODEL = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    # base_url=TGI_BASE_URL,
    model=TGI_MODEL,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput


class Result(BaseAssistantInput):
    result: int

In [14]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


request = LLMRequest(
    conversation=LLMConversation(
        messages=[
            LLMMessage(role='system', content='You are a helpful assistant.'),
            LLMMessage(role='user', content='what is 2+2'),
        ]
    ),
    params=LLMParams(
        model=TGI_MODEL, temperature=0, response_type=Result, max_completion_tokens=10
    ),
)
batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]

lines = [json.dumps(request, separators=(',', ':')) for request in requests]

In [15]:
request_dict = json.loads(lines[0])

In [17]:
result = await client.chat_completion(**request_dict)  # TODO
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1743597261, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [ ]:
response_list = LLMResponseListMapping.to_source(
    result, request_id=..., input_id=..., response_type=...
)

In [ ]:
input_file_path = Path(f'input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

In [ ]:
result = await client.chat_completion(
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'what is 2+2'},
    ],
    model=TGI_MODEL,
    response_format=None,
    temperature=0,
    max_tokens=10,
)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='length', index=0, message=ChatCompletionOutputMessage(role='assistant', content='The sum of 2 and 2 is ', tool_call_id=None, tool_calls=None), logprobs=None)], created=1743595756, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=17, total_tokens=27), object='chat.completion')

In [ ]:
result = await client.chat_completion(
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'what is 2+2'},
    ],
    model=TGI_MODEL,
    response_format={
        'type': 'json',  # json or regex
        'value': Result.model_json_schema(),  # json schema
    },
    temperature=0,
    max_tokens=10,
)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1743595874, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')